# 并查集 (Disjoint Set Union) 详解

## 一、 什么是并查集？

并查集（Disjoint Set Union, DSU），也常被称为联合-查找算法（Union-Find Algorithm），是一种用于处理不相交集合（Disjoint Sets）的合并与查询问题的数据结构。

想象一下管理“朋友圈”的场景：
1.  初始时，每个人都是一个独立的朋友圈。
2.  当两个人成为朋友时，他们各自的朋友圈就合并成了一个更大的朋友圈。
3.  我们随时可能需要查询：某两个人是否在同一个朋友圈里？

并查集就是为了高效解决这类问题而生的。它主要支持两种核心操作：
* **Union (合并)**：将两个不相关的集合合并成一个。
* **Find (查找)**：确定一个元素属于哪个集合，通常是通过查找该集合的“代表”或“根节点”。

## 二、 实现原理：树形表示

并查集最经典的实现方式是使用一个数组来模拟一个由多棵树组成的“森林”，其中每一棵树代表一个集合。

我们用一个 `parent` 数组来维护这个森林：
* `parent[i]` 的值表示元素 `i` 的父节点。
* **根节点（代表）**：如果一个元素的父节点是它自己，即 `parent[i] == i`，那么它就是这棵树的根节点，也是其所在集合的唯一代表。

### 1. 初始化

开始时，每个元素都自成一个集合。因此，我们将每个元素的父节点都设置为它自己。

```Python
# n 为元素总数
# 假设 n = 5
parent = [0, 1, 2, 3, 4]
```

### 2. Find 操作 (查找)

`Find(i)` 操作就是从元素 `i` 开始，沿着它的父节点不断向上回溯，直到找到根节点为止。

**朴素实现：**
```Python
def find_naive(parent, i):
    while parent[i] != i:
        i = parent[i]
    return i
```
**问题**：如果树的结构退化成一条长链，例如 `3 -> 2 -> 1 -> 0`，那么每次 `find(3)` 都需要遍历整条链，时间复杂度为 $O(N)$，效率很低。

### 3. Union 操作 (合并)

`Union(i, j)` 操作是合并元素 `i` 和 `j` 所在的两个集合。
1.  首先，通过 `Find` 操作找到 `i` 和 `j` 各自的根节点 `root_i` 和 `root_j`。
2.  如果 `root_i` 和 `root_j` 不相同（说明它们不在同一个集合），就将一棵树的根节点连接到另一棵树的根节点上，例如 `parent[root_i] = root_j`。

**朴素实现：**
```Python
def union_naive(parent, i, j):
    root_i = find_naive(parent, i)
    root_j = find_naive(parent, j)
    if root_i != root_j:
        parent[root_i] = root_j
```

## 三、 关键优化

为了解决朴素实现中效率低下的问题，我们必须引入两个至关重要的优化。

### 1. 路径压缩 (Path Compression)

这是 `Find` 操作的优化。当我们在从一个节点向上走到根节点的途中，我们将路径上经过的**所有节点**的父指针**直接指向根节点**。

这样做的好处是极大地“压平”了树的高度。下次再查找这条路径上的任何节点时，都可以一步到位，时间复杂度接近 $O(1)$。

**实现 (递归版):**
```Python
def find(parent, i):
    # 如果i不是根节点
    if parent[i] != i:
        # 递归地查找根节点，并将i的父节点直接设为根节点
        parent[i] = find(parent, parent[i])
    return parent[i]
```

### 2. 按秩合并 (Union by Rank / Size)

这是 `Union` 操作的优化，目的是在合并时让树尽可能地“矮”。我们为每个根节点维护一个“秩”（rank），它可以是树的高度（Union by Rank）或集合的大小（Union by Size）。

**规则**：在合并两棵树时，总是将**秩较小**的树的根节点，连接到**秩较大**的树的根节点上。

* **按大小合并 (Union by Size)** 更为常用且简单有效。我们用一个 `size` 数组记录以每个根节点为首的集合的大小。合并时，小集合并入大集合，并更新大集合的大小。

这样做可以有效地防止树的高度无限制地增长，使其保持相对平衡。

## 四、 完整 Python 实现

结合了**路径压缩**和**按大小合并**的并查集是效率最高的版本。

```Python
class DSU:
    """
    一个实现了路径压缩和按大小合并的并查集类
    """
    def __init__(self, n):
        """
        初始化n个元素，每个元素自成一个集合。
        """
        # parent[i] 表示元素i的父节点
        self.parent = list(range(n))
        # size[i] 表示以i为根的集合的大小（只对根节点有效）
        self.size = [1] * n
        # 连通分量的数量
        self.count = n

    def find(self, i):
        """
        查找元素i的根节点，并在此过程中进行路径压缩。
        """
        if self.parent[i] == i:
            return i
        
        # 递归查找根节点，并将路径上所有节点的父节点直接指向根节点
        self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        """
        合并元素i和j所在的集合。
        如果它们已经在同一个集合，则不操作并返回False。
        否则，执行合并并返回True。
        """
        root_i = self.find(i)
        root_j = self.find(j)
        
        if root_i != root_j:
            # 按大小合并：将小集合合并到大集合
            if self.size[root_i] < self.size[root_j]:
                # 保证 root_i 是较大集合的根
                root_i, root_j = root_j, root_i
            
            # 将小集合的根(root_j)连接到大集合的根(root_i)
            self.parent[root_j] = root_i
            # 更新大集合的大小
            self.size[root_i] += self.size[root_j]
            # 连通分量数量减1
            self.count -= 1
            return True
            
        return False

# --- 使用示例 ---
# 假设有10个城市，编号0-9
num_cities = 10
dsu = DSU(num_cities)

# 修建一些道路
dsu.union(0, 1)
dsu.union(1, 2)
dsu.union(3, 4)
dsu.union(5, 6)
dsu.union(6, 7)
dsu.union(8, 9)

# 查询城市间的连通性
print(f"城市 0 和 2 是否连通? {'是' if dsu.find(0) == dsu.find(2) else '否'}")
print(f"城市 0 和 4 是否连通? {'是' if dsu.find(0) == dsu.find(4) else '否'}")
print(f"当前的连通分量数量: {dsu.count}")

# 再修建一条路，连接两个不同的区域
print("\n修建道路连接城市 2 和 3...")
dsu.union(2, 3)

print(f"现在城市 0 和 4 是否连通? {'是' if dsu.find(0) == dsu.find(4) else '否'}")
print(f"当前的连通分量数量: {dsu.count}")
```

## 五、 时间复杂度

当同时使用**路径压缩**和**按秩/大小合并**优化时，`find` 和 `union` 操作的平均时间复杂度可以达到**近乎常数级别**，记为 $O(\alpha(N))$。

这里的 $\alpha(N)$ 是反阿克曼函数，它增长得极其缓慢。对于在宇宙中能想象到的任何大小的 $N$，$\alpha(N)$ 的值都不会超过 5，因此在实践中完全可以将其视为一个常数。

必须同时使用两种优化才能达到，如果只用其中一种，复杂度会变差（比如只用路径压缩，最坏情况是 O(logn)）。

## 六、 经典应用

1.  **最小生成树 (Minimum Spanning Tree)**：在 [Kruskal 算法](https://zh.wikipedia.org/wiki/%E5%85%8B%E9%B2%81%E6%96%AF%E5%85%8B%E5%B0%94%E7%AE%97%E6%B3%95)中，用于快速判断加入一条新的边是否会形成环路。
2.  **判断图的连通性**：计算一个无向图中有多少个连通分量，或者判断任意两个顶点是否连通。
3.  **图像处理**：用于标记和计算图像中像素的连通区域。
4.  **词法分析**：在一些编译器或解释器的实现中，用于处理等价性关系。

# 笔记：并查集与数学等价关系的深刻联系

## 摘要

并查集（Disjoint Set Union, DSU）并非一个孤立的计算机算法，它在数学上有着坚实而优美的理论基础。**本质上，并查集是为高效计算和动态维护“等价关系”而设计的算法实现。** 理解这一点，能帮助我们从更深层次认识并查集的应用场景和威力。

---

## 1. 数学回顾：什么是等价关系？

在数学中，一个集合 `S` 上的“关系” `~` 如果满足以下三个核心性质，就被称为 **等价关系** (Equivalence Relation)。

#### 等价关系三公理：

1.  **自反性 (Reflexivity)**
    * 对于集合 `S` 中的任何元素 `a`，都有 `a ~ a`。

2.  **对称性 (Symmetry)**
    * 如果 `a ~ b` 成立，那么 `b ~ a` 也必然成立。

3.  **传递性 (Transitivity)**
    * 如果 `a ~ b` 且 `b ~ c` 都成立，那么 `a ~ c` 也必然成立。

#### 核心推论：划分等价类

一个集合上的等价关系最重要的作用，是它能将这个集合 **划分 (Partition)** 成若干个互不相交的子集。每一个子集被称为一个 **等价类 (Equivalence Class)**。

* **类内皆等价**：同一个等价类中的所有元素彼此都是等价的。
* **类间不等价**：不同等价类中的元素是不等价的。

**示例**：
* **集合 S**：{北京, 上海, 广州, 东京, 巴黎, 罗马}
* **等价关系 ~**：“属于同一个国家”。
* **等价类划分**：
    * `{北京, 上海, 广州}` (中国)
    * `{东京}` (日本)
    * `{巴黎}` (法国)
    * `{罗马}` (意大利)

这四个等价类互不相交，它们的并集恰好是完整的集合 S。

---

## 2. 桥梁：并查集如何完美建模等价关系

并查集的每一个组成部分和操作，都与等价关系的概念一一对应。它将抽象的数学定义转化为了具体的数据结构和算法操作。

| 数学概念 (Mathematical Concept) | 并查集实现 (DSU Implementation) | 解释 |
| :--- | :--- | :--- |
| **集合 `S` 中的一个元素 `a`** | **并查集中的一个节点 `a`** | 并查集中的每个初始节点都代表原始集合中的一个元素。 |
| **一个等价类** | **一棵树 / 一个连通分量** | 并查集中的每一棵树代表一个等价类。树中的所有节点都属于同一个等价类。 |
| **等价类的唯一标识/代表** | **树的根节点 (Root)** | `find(a)` 操作正是为了查找元素 `a` 所属等价类的那个唯一代表。 |
| **判断 `a` 和 `b` 是否等价 (`a ~ b`)** | **`find(a) == find(b)`** | 如果两个元素的根节点相同，说明它们在同一棵树里，即属于同一个等价类